### In this file, we will generate our E0 feature data

Instructions: 

You should start by writing a notebook to create the dataset. Note that if you use many molecules, using MOPAC could take a while. If you decide to optimize the geometry of all the molecules in the Solubility datasets, ot will take at least 12 hours. Make sure to save the result so that you don't have to compute it multiple times!

In [ ]:
# Pandas dataframe, apply function to whole dataframe
# Make sure it works on a small dataset. Save the results
# Create dataset associated smiles to energy. Then, run overnight on all data
# Once dataset ready, can move on to next step